<a href="https://colab.research.google.com/github/caiojgouvea/crudCompletoFlutter/blob/main/Covid_Algoritimos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports necessários

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn import linear_model
from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
from fbprophet import Prophet
# import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [ ]:
ds_covid_br = pd.read_csv("./drive/MyDrive/TCC - Anna, Caio, Igor/05 - Dataset/caso_full.csv", delimiter=',')
ds_covid_br.head()

,city,city_ibge_code,date,epidemiological_week,estimated_population,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,last_available_death_rate,last_available_deaths,order_for_place,place_type,state,new_confirmed,new_deaths
0,Rio Branco,1200401.0,2020-03-17,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-17,0.0,0,1,city,AC,3,0
1,NaN,12.0,2020-03-17,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-17,0.0,0,1,state,AC,3,0
2,Rio Branco,1200401.0,2020-03-18,202012,413418.0,407319.0,False,False,3,0.72566,2020-03-18,0.0,0,2,city,AC,0,0
3,NaN,12.0,2020-03-18,202012,894470.0,881935.0,False,False,3,0.33539,2020-03-18,0.0,0,2,state,AC,0,0
4,Rio Branco,1200401.0,2020-03-19,202012,413418.0,407319.0,False,False,4,0.96754,2020-03-19,0.0,0,3,city,AC,1,0


In [ ]:
# Criar dataframe apenas com os dados de Goiás
ds_estados = ds_covid_br.loc[ds_covid_br.place_type=='state',:]
ds_estado_go = ds_estados.loc[ds_covid_br.state=='GO',:]
ds_estado_go_casos = ds_estado_go.groupby('date').sum()['new_deaths'].reset_index()
ds_estado_go_casos['date'] = pd.to_datetime(ds_estado_go_casos['date'])

In [ ]:
# Prever as novas mortes confirmadas
attr = ['date']
attr_prev = ['new_deaths']

# Criando objetos
X = ds_estado_go_casos[attr].values
y = ds_estado_go_casos[attr_prev].values

# Validação
validate = 0.30

# Criando dados de treino e validação
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = validate)

# Utilizar o RandonForestRegressor para predizer quantas novas mortes teremos

In [ ]:
# Treinar o modelo
modelo = RandomForestRegressor(n_estimators=1000, max_depth=30)
modelo.fit(X_treino, y_treino.ravel())

# Predizer o modelo
predicao = modelo.predict(np.array(pd.to_datetime(['2020-07-07', '2020-07-08', '2020-07-09'])).reshape(-1,1))
# print(predicao)
modelo.score(X_treino, y_treino) * 100

88.36911399731683

# Utilizar o GradientBoostingRegressor para predizer quantas novas mortes teremos

In [ ]:
from sklearn.metrics import mean_squared_error

modeloGradient = GradientBoostingRegressor(n_estimators=100, max_depth=20)
modeloGradient.fit(X_treino, y_treino.ravel())

resultado = modeloGradient.predict(np.array(pd.to_datetime(['2020-07-07', '2020-07-08', '2020-07-09'])).reshape(-1, 1))

print(resultado)

[33.00021355 46.99984169 46.99984169]


In [ ]:
# Treinar o modelo



# Essemble do RandomForest
modeloXGBoost = GradientBoostingRegressor(n_estimators=1000, max_depth=2)
modeloXGBoost.fit(X_treino, y_treino.ravel())

# Predizer o modelo
predicao = modeloXGBoost.predict(np.array(pd.to_datetime(['2020-07-07', '2020-07-08', '2020-07-09'])).reshape(-1,1))
# print(predicao)
print(predicao)


[31.60590034 31.60590034 31.60590034]


# Utilizar o LinearRegression para predizer quantas novas mortes teremos

In [ ]:
# import datetime as dt

# modeloLR = LinearRegression()
# modeloLR.fit(X_treino, y_treino)

# X_treino2 = pd.DataFrame(np.array(X_treino, dtype='datetime64[ns]'))
# y_treino2 = pd.DataFrame(np.array(y_treino, dtype='datetime64[ns]'))

# X_treino2.columns = ["date"]
# X_treino2['date'] = pd.to_datetime(X_treino2['date'])
# X_treino2['date'] = X_treino2['date'].map(dt.datetime.toordinal)

# y_treino2.columns = ["date"]
# y_treino2['date'] = pd.to_datetime(y_treino2['date'])
# y_treino2['date'] = y_treino2['date'].map(dt.datetime.toordinal)

# predicao = modeloLR.predict(pd.to_datetime(['2021-12-25']).map(dt.datetime.toordinal)[0].reshape(-1,1))

# modeloLR.score(X_treino2, y_treino2)
# print(predicao)

# Utilizar o Prophet
## https://facebook.github.io/prophet/

Fazer a implementação do Prophet

Criar a API flask

In [ ]:
# Criar modelo Prophet
ds_estados = ds_covid_br.loc[ds_covid_br.place_type=='state',:]
ds_estado_go = ds_estados.loc[ds_covid_br.state=='GO',:]
ds_estado_go_casos = ds_estado_go.groupby('date').sum()['new_deaths'].reset_index()

ds_estado_go_casos_previsao = ds_estado_go_casos
ds_estado_go_casos_previsao.columns = ['ds', 'y']

modeloPH = Prophet(yearly_seasonality = False, daily_seasonality = False)

modeloPH.fit(ds_estado_go_casos_previsao)

# Criar um dataframe futuro
future = modeloPH.make_future_dataframe(60)

# Prever os novos dados
previsao = modeloPH.predict(future)

# Apresentar apenas as ultimas 5 linhas
previsao[['ds', 'yhat_lower', 'yhat', 'yhat_upper']].tail(5)

,ds,yhat_lower,yhat,yhat_upper
628,2021-11-30,-6.249584,38.890240,86.343563
629,2021-12-01,-3.027717,42.050246,86.286517
630,2021-12-02,-2.882740,44.535312,90.627958
631,2021-12-03,-13.170311,35.243477,80.387684
632,2021-12-04,-35.907253,9.692721,56.236627


In [ ]:
from pandas import DataFrame
from pandas import to_datetime

# Criar modelo Prophet
ds_estados = ds_covid_br.loc[ds_covid_br.place_type=='state',:]
ds_estado_go = ds_estados.loc[ds_covid_br.state=='GO',:]
ds_estado_go_casos = ds_estado_go.groupby('date').sum()['new_deaths'].reset_index()

ds_estado_go_casos_previsao = ds_estado_go_casos
ds_estado_go_casos_previsao.columns = ['ds', 'y']

modeloProphet = Prophet(yearly_seasonality = False, daily_seasonality = False)

modeloProphet.fit(ds_estado_go_casos_previsao)

# Criar um dataframe futuro
future = ['2020-07-07', '2020-07-08', '2020-07-09']
future = DataFrame(future)
future.columns = ['ds']
future['ds'] = to_datetime(future['ds'])

# predicao = modeloXGBoost.predict(np.array(pd.to_datetime(['2020-07-07', '2020-07-08', '2020-07-09'])).reshape(-1,1))
previsao = modeloProphet.predict(future)
dados = previsao['yhat_upper'].to_numpy()
print(previsao[['ds', 'yhat_lower', 'yhat', 'yhat_upper']])

          ds  yhat_lower       yhat  yhat_upper
0 2020-07-07  -22.009197  27.617996   74.196301
1 2020-07-08  -19.126740  31.130809   76.970626
2 2020-07-09  -10.832685  33.968683   82.679465


In [ ]:
import pickle

filename = 'modeloGradient.pkl'
pickle.dump(modeloGradient, open(filename, 'wb'))



# **Criar API**

## Api para cada algoritimo

https://medium.com/data-hackers/disponibilizando-o-seu-modelo-de-machine-learning-em-produ%C3%A7%C3%A3o-com-python-flask-e-docker-aa4c43c35b50